<a href="https://colab.research.google.com/github/achmadbauravindah/Chatbot-TFIDF-CancerArticle/blob/main/ChatBot_TFIDF_CancerArticle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import nltk
import string
import random
import requests
nltk.download('popular', quiet=True) # for downloading packages

True

In [ ]:
# Get Document
URL = "https://raw.githubusercontent.com/AchmadBauravindah/datas_for_ai/main/nlp/document_for_chatbot_answers.txt"
req = requests.get(URL)
document = req.text

In [ ]:
# Make document to lowercase words
document = document.lower()

In [ ]:
# Make sentences and word corpus
sentence_tokens = nltk.sent_tokenize(document)
word_tokens = nltk.word_tokenize(document)

In [ ]:
# Example of Sentences Tokens
sentence_tokens[:5]

['this article is about the group of diseases.',
 'for other uses, see cancer (disambiguation).',
 'cancer\nother names\tmalignant tumor, malignant neoplasm\ntumor mesothelioma2 legend.jpg\na coronal ct scan showing a malignant mesothelioma\nlegend: → tumor ←, ✱ central pleural effusion, 1 & 3 lungs, 2 spine, 4 ribs, 5 aorta, 6 spleen, 7 & 8 kidneys, 9 liver\npronunciation\t\n/ˈkænsər/ (listen)\nspecialty\toncology\nsymptoms\tlump, abnormal bleeding, prolonged cough, unexplained weight loss, change in bowel movements[1]\nrisk factors\texposure to carcinogens, tobacco, obesity, poor diet, lack of physical activity, excessive alcohol, certain infections[2][3]\ntreatment\tradiation therapy, surgery, chemotherapy, and targeted therapy.',
 '[2][4]\nprognosis\taverage five year survival 66% (usa)[5]\nfrequency\t24 million annually (2019)[6]\ndeaths\t10 million annually (2019)[6]\ncancer is a group of diseases involving abnormal cell growth with the potential to invade or spread to other part

In [ ]:
# Example of Word Tokens 
word_tokens[:5]

['this', 'article', 'is', 'about', 'the']

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# Lemmatization for every words
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer() # WordNet Lemmatizer adalah untuk konversi kata-kata menjadi kata dasar seperti cars -> car dan tidak berlaku jika am -> be
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation) # Membuat dictionary punctuation menjadi None
def LemTokens(tokens):
  return [wnl.lemmatize(token) for token in tokens]
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
# Make Greeting for Bot
GREET_INPUTS = ("hello", "hi", "greetings", "what's up", "hey")
GREET_RESPONSES = ["hi", "hey", "hi there", "hello", "I am glad! You are talking to me"]

def greet(sentence):
  for word in sentence.split():
    if word.lower() in GREET_INPUTS:
      return random.choice(GREET_RESPONSES)

In [ ]:
# Important library for BOT Response
from sklearn.feature_extraction.text import TfidfVectorizer # Membuat vektor beserta nilai tf-idf
from sklearn.metrics.pairwise import cosine_similarity # Mencari jarak atau nilai paling sama pada hasil tf-idf

In [ ]:
# BOT Response
def response(user_response):
  bot_response = ""
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize) # Memakai tokenizer yang dibuat sediri
  tfidf_results = TfidfVec.fit_transform(sentence_tokens) # Mnedapatkan vektor atau nilai tf-idf
  word_similar = cosine_similarity(tfidf_results[-1], tfidf_results) # Membandingkan kemiripan user_response dan tfidf_results
  idx = word_similar.argsort()[0][-2] # Mencari index kedua terakhir karena yang paling mirip dengan user_response (input sentence) [0]: mengambil array pertama (word_similar array memang hanya 1) [-2]: Mengambil index kedua terakhir karena yang terakhir adalah user_response
  word_similar_to_flat = word_similar.flatten()
  word_similar_to_flat.sort()
  req_tfidf = word_similar_to_flat[-2] # Mengambil nilai tf-idf pada index terakhir (karena yang paling besar sehingga bisa dikatakan yang paling mirip)
  if (req_tfidf == 0):
    bot_response = bot_response + "I am sorry! I don't understand you"
    return bot_response
  else:
    bot_response = bot_response + sentence_tokens[idx] # sentence_tokens[idx]: mengambil sentence asli pada index yang sudah dihitung tadi yang paling mirip
    return bot_response

In [ ]:
# BOT Conversation
talk = True
print("BOT: Hello, can i help you?. Please type 'Bye' if you exit 🙏")
while (talk == True):
  user_response = input()
  user_response = user_response.lower()
  if (user_response !=  'bye'):
    if (user_response == 'thanks' or user_response == 'thank you'):
      talk = False
      print("BOT: You're Welcome")
    else:
      if (greet(user_response) != None):
        print("BOT: " + greet(user_response))
      else:
        sentence_tokens.append(user_response)
        word_tokens = word_tokens + nltk.word_tokenize(user_response)
        # final_words = list(set(word_tokens))
        print("BOT: ", end="")
        print(response(user_response))
        sentence_tokens.remove(user_response)
  else:
    talk = False
    print("BOT: Good bye! Have a nice day")

BOT: Hello, can i help you?. Please type 'Bye' if you exit 🙏
Bye
BOT: Good bye! Have a nice day


# Kesimpulan

Proses utama dalam membuat sistem adalah:
1. Query Document
2. Pre-Processing Text
3. Processing Text
4. Bot System

### 1. Query Document
- Dokumen diambil dari GitHub
- Dokumen berisi artikel atau informasi mengenai topik tertentu.
- Dokumen digunakan oleh bot untuk mencari jawaban dari pertanyaan user
- Tanpa dokumen bot tidak bisa menjawab pertanyaan dengan baik
- Semakin banyak dokumen maka bot semakin pintar

### 2. Pre-Processing Text
- Lowercase
- Tokenize text perkalimat dan perkata
- Membuang tanda baca
- Vectorization

### 3. Processing Text
- Menghitung nilai TF-IDF pada dokumen, TF-IDF dihitung berdasarkan kalimat pada dokumen tersebut. TF-IDF memberikan nilai pada setiap kalimat.

- Cosine Similar, dilakukan untuk mencari nilai terdekat atau paling mirip antara “user input” dan “kalimat pada dokumen”

### 4. Bot Systems

- BOT Greeting (Sapaan dari BOT)
- BOT Response, mengatur semua proses yang dilakukan secara berurutan seperti user memasukan input, kalimat user di pre-process dan bot memberikan respon dari hasil cosine similar dari kalimat pertanyaan input user


# Perubahan Modifikasi

### 1. Modifkasi Data
Data yang sebelumnya artikel mengenai chatbot, maka saya ganti menjadi artikel cancer yang say ambil dari wikipedia (https://en.wikipedia.org/wiki/Cancer), artikel ini langsung saya copy mulai dari baris awal sampai akhir tanpa memperhatikan apapun, karena didalam sistem ini sudah ada preprocess

### 2. Modifikasi Variabel
Ada beberapa variabel yang membingungkan seperti "sent_tokens" saya rubah menjadi "sentence_tokens", hal ini dikarenakan arti dari "sent" adalah kirim, saya mulanya mencari tahu, kenapa harus "sent" ternyata yang dimaksud adalah "sentences",

# Adapun Kekurangan dalam sistem ini adalah:
- Tidak ada remove stopwords pada saat preprocessing
- Proses stemming tidak dilakukan (Hanya lemmatization saja)
- TF-IDF masih menggunakan library
- Pre-processing penghilangan kata "[60]" atau "[48]" atau yang lain masih ada (bisa dilihat pada output Chatbots)